### 전복데이터의 분류/회귀를 CNN과 LSTM으로 분류 및 회귀를 작성하라

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, LSTM, Dropout, Input
from tensorflow.keras.optimizers import Adam

In [2]:
url = "C:/Users/dnwjd/OneDrive/Desktop/CSE_6/딥러닝/AI-class-main/AI-class-main/abalone.csv"
data = pd.read_csv(url)

In [3]:
print(data.head())

   id Sex  Length  Diameter  Height  Whole_weight  Shucked_weight  \
0   0   M   0.455     0.365   0.095        0.5140          0.2245   
1   1   M   0.350     0.265   0.090        0.2255          0.0995   
2   2   F   0.530     0.420   0.135        0.6770          0.2565   
3   3   M   0.440     0.365   0.125        0.5160          0.2155   
4   4   I   0.330     0.255   0.080        0.2050          0.0895   

   Viscera_weight  Shell_weight  Rings  
0          0.1010         0.150     15  
1          0.0485         0.070      7  
2          0.1415         0.210      9  
3          0.1140         0.155     10  
4          0.0395         0.055      7  


In [4]:
label_encoder = LabelEncoder()
data['Sex'] = label_encoder.fit_transform(data['Sex'])

In [5]:
data['Ring_Class'] = (data['Rings'] >= data['Rings'].median()).astype(int)  # 중간값 기준 이진 분류

In [6]:
# 특징과 레이블 분리
X_class = data.drop(['Rings', 'Ring_Class'], axis=1)
y_class = data['Ring_Class']

X_reg = data.drop(['Rings'], axis=1)
y_reg = data['Rings']

In [7]:
# 데이터셋 분할
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.2, random_state=42)
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.2, random_state=42)

In [8]:
# 데이터 표준화
scaler_class = StandardScaler()
X_train_class = scaler_class.fit_transform(X_train_class)
X_test_class = scaler_class.transform(X_test_class)

scaler_reg = StandardScaler()
X_train_reg = scaler_reg.fit_transform(X_train_reg)
X_test_reg = scaler_reg.transform(X_test_reg)

In [9]:
# CNN과 LSTM 모델을 위한 데이터 형태 변환 (3D 입력으로 변환)
X_train_class_cnn = X_train_class.reshape(X_train_class.shape[0], X_train_class.shape[1], 1)
X_test_class_cnn = X_test_class.reshape(X_test_class.shape[0], X_test_class.shape[1], 1)

X_train_reg_cnn = X_train_reg.reshape(X_train_reg.shape[0], X_train_reg.shape[1], 1)
X_test_reg_cnn = X_test_reg.reshape(X_test_reg.shape[0], X_test_reg.shape[1], 1)

In [11]:
# CNN 분류 모델
cnn_model_class = Sequential([
    Conv1D(32, 2, activation='relu', input_shape=(X_train_class_cnn.shape[1], 1)),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # 이진 분류
])

cnn_model_class.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
cnn_model_class.fit(X_train_class_cnn, y_train_class, epochs=20, batch_size=32, validation_split=0.2)
loss, accuracy = cnn_model_class.evaluate(X_test_class_cnn, y_test_class)
print(f"\n=== CNN Classifier ===\nTest Accuracy: {accuracy:.4f}")

Epoch 1/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 3s 7ms/step - accuracy: 0.8087 - loss: 0.4904 - val_accuracy: 0.8161 - val_loss: 0.3916
Epoch 2/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8383 - loss: 0.3844 - val_accuracy: 0.8296 - val_loss: 0.3703
Epoch 3/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8334 - loss: 0.4027 - val_accuracy: 0.8401 - val_loss: 0.3592
Epoch 4/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8418 - loss: 0.3633 - val_accuracy: 0.8430 - val_loss: 0.3526
Epoch 5/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8352 - loss: 0.3689 - val_accuracy: 0.8386 - val_loss: 0.3488
Epoch 6/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8497 - loss: 0.3413 - val_accuracy: 0.8430 - val_loss: 0.3476
Epoch 7/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8444 - loss: 0.3490 - val_accuracy: 0.8356 - val_loss: 0.3480
Epoch 8/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8289 - loss: 0.3539 - val_accuracy: 0.8386 - val_loss:

In [12]:
# CNN 회귀 모델
cnn_model_reg = Sequential([
    Conv1D(32, 2, activation='relu', input_shape=(X_train_reg_cnn.shape[1], 1)),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)  # 회귀
])

In [13]:
cnn_model_reg.compile(optimizer=Adam(), loss='mse', metrics=['mse'])
cnn_model_reg.fit(X_train_reg_cnn, y_train_reg, epochs=20, batch_size=32, validation_split=0.2)
loss, mse = cnn_model_reg.evaluate(X_test_reg_cnn, y_test_reg)
print(f"\n=== CNN Regressor ===\nTest Mean Squared Error: {mse:.4f}")

Epoch 1/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 74.6290 - mse: 74.6290 - val_loss: 13.6120 - val_mse: 13.6120
Epoch 2/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 19.1889 - mse: 19.1889 - val_loss: 8.7764 - val_mse: 8.7764
Epoch 3/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 9.8041 - mse: 9.8041 - val_loss: 6.7028 - val_mse: 6.7028
Epoch 4/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7.1013 - mse: 7.1013 - val_loss: 5.7344 - val_mse: 5.7344
Epoch 5/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.2151 - mse: 6.2151 - val_loss: 5.3849 - val_mse: 5.3849
Epoch 6/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.4067 - mse: 6.4067 - val_loss: 5.0891 - val_mse: 5.0891
Epoch 7/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.0601 - mse: 6.0601 - val_loss: 4.9177 - val_mse: 4.9177
Epoch 8/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.5972 - mse: 5.5972 - val_loss: 4.7848 - val_mse: 4.7848
Epoch 9/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.3933 -

In [14]:
# LSTM 분류 모델
lstm_model_class = Sequential([
    LSTM(64, activation='relu', input_shape=(X_train_class_cnn.shape[1], 1)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # 이진 분류
])

C:\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
lstm_model_class.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
lstm_model_class.fit(X_train_class_cnn, y_train_class, epochs=20, batch_size=32, validation_split=0.2)
loss, accuracy = lstm_model_class.evaluate(X_test_class_cnn, y_test_class)
print(f"\n=== LSTM Classifier ===\nTest Accuracy: {accuracy:.4f}")

Epoch 1/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.7707 - loss: 0.5618 - val_accuracy: 0.8296 - val_loss: 0.3886
Epoch 2/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8201 - loss: 0.4095 - val_accuracy: 0.8326 - val_loss: 0.3986
Epoch 3/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.8235 - loss: 0.4016 - val_accuracy: 0.8371 - val_loss: 0.3809
Epoch 4/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8291 - loss: 0.3909 - val_accuracy: 0.8401 - val_loss: 0.3788
Epoch 5/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8206 - loss: 0.4023 - val_accuracy: 0.8430 - val_loss: 0.3744
Epoch 6/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8382 - loss: 0.3736 - val_accuracy: 0.8445 - val_loss: 0.3737
Epoch 7/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8266 - loss: 0.3896 - val_accuracy: 0.8445 - val_loss: 0.3753
Epoch 8/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8031 - loss: 0.4207 - val_accuracy: 0.8445 - val_loss

In [16]:
# LSTM 회귀 모델
lstm_model_reg = Sequential([
    LSTM(64, activation='relu', input_shape=(X_train_reg_cnn.shape[1], 1)),
    Dense(32, activation='relu'),
    Dense(1)  # 회귀
])

In [17]:
lstm_model_reg.compile(optimizer=Adam(), loss='mse', metrics=['mse'])
lstm_model_reg.fit(X_train_reg_cnn, y_train_reg, epochs=20, batch_size=32, validation_split=0.2)
loss, mse = lstm_model_reg.evaluate(X_test_reg_cnn, y_test_reg)
print(f"\n=== LSTM Regressor ===\nTest Mean Squared Error: {mse:.4f}")

Epoch 1/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - loss: 86.8168 - mse: 86.8168 - val_loss: 21.6790 - val_mse: 21.6790
Epoch 2/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 12.8582 - mse: 12.8582 - val_loss: 8.1113 - val_mse: 8.1113
Epoch 3/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 8.1226 - mse: 8.1226 - val_loss: 6.3326 - val_mse: 6.3326
Epoch 4/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 7.5006 - mse: 7.5006 - val_loss: 5.9786 - val_mse: 5.9786
Epoch 5/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 5.8448 - mse: 5.8448 - val_loss: 6.9275 - val_mse: 6.9275
Epoch 6/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 6.2771 - mse: 6.2771 - val_loss: 5.4005 - val_mse: 5.4005
Epoch 7/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 6.2750 - mse: 6.2750 - val_loss: 5.1791 - val_mse: 5.1791
Epoch 8/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 6.7407 - mse: 6.7407 - val_loss: 5.1037 - val_mse: 5.1037
Epoch 9/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 5.8734 